In [1]:
from sequence import *
from sym import *
from sstruct import *

#### Chou-Fasman table  
cf_dict = {   
     (H),  P(E),  P(T),  f(i), f(i+1), f(i+2),  f(i+3)  
'A': ( 142,   83,   66, 0.060,  0.076,  0.035,  0.058 ),    # Alanine  
'R': (  98,   93,   95, 0.070,  0.106,  0.099,  0.085 ),    # Arginine  
...,  
'V': ( 106,  170,   50, 0.062,  0.048,  0.028,  0.053 ),}   # Valine  

The first three scores for each amino acid correspond to its tendency to be involved in **alpha-helices (H), beta-strands (E), and turns (T)**, respectively.

The two files **prot2.fa and sstr3.fa** should refer to the same set of proteins stored as amino acid sequence and secondary structure sequence respectively.

In [2]:
# read both protein and structure sequences into lists
prot2 = readFastaFile("prot2.fa", Protein_Alphabet)
sstr3 = readFastaFile("sstr3.fa", DSSP3_Alphabet)

#### Predicting helical structures with the markCountAbove functionOutline
The Chou-Fasman rules are based on windows of amino acids and their scores, as secondary structure generally depends on multiple amino acids occurring together.
To identify alpha-helices we look at 6 consecutive residues at a time and check if 4 or more have an alpha score of at least 100.

In [3]:
above = markCountAbove(markCountAboveTest)
print (above)

[False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False]


#### Exercise 3 : Apply your functions to the PDB entry 2NLU
The below protein sequence is the C-terminus of the PDB entry 2NLU starting at position 76 and onwards. Consider the helical secondary structure of its amino acid sequence PNKRKGFSEGLWEIENNPTVKASGY, which we can generate with in Python, like so:

In [4]:
myprot = Sequence('PNKRKGFSEGLWEIENNPTVKASGY', Protein_Alphabet, '2NLU_r76')
alpha = getScores(myprot, 0)
calls_a1 = markCountAbove(alpha, width = 6, call_cnt = 4)
print("the index of getScores is 0: " + makesstr(calls_a1, 'H'))

alpha = getScores(myprot, 1)
calls_a1 = markCountAbove(alpha, width = 6, call_cnt = 4)
print("the index of getScores is 1: " + makesstr(calls_a1, 'H'))

alpha = getScores(myprot, 2)
calls_a1 = markCountAbove(alpha, width = 6, call_cnt = 4)
print("the index of getScores is 2: " + makesstr(calls_a1, 'H'))

the index of getScores is 0: -HHHHHHHHHHHHHHHHHHHHH---
the index of getScores is 1: -------------------------
the index of getScores is 2: HHHHHHHHHH-----HHHHHHHHHH


### Exercise 4 : Scripting the full Chou-Fasman algorithm

the process of applying the Chou-Fasman prediction rules for alpha-helices and beta-strands

In [5]:
print("Steps 1 to 4")
# Step 1: Retrieve scores for each amino acid
alpha = getScores(myprot, 0) 
beta = getScores(myprot, 1)

# Step 2: Alpha-helix 
calls_a1 = markCountAbove(alpha, width = 6, call_cnt = 4) 
print(makesstr(calls_a1, 'H'))

# Step 3: Alpha-helix extension
calls_a2 = extendDownstream(alpha, calls_a1, width = 4) 
calls_a3 = extendUpstream(alpha, calls_a2, width = 4)

# Step 4: Beta-sheet and extension
calls_b1 = markCountAbove(beta, width = 5, call_cnt = 3) 
print(makesstr(calls_b1, 'E'))
calls_b2 = extendDownstream(beta, calls_b1, width = 4) 
calls_b3 = extendUpstream(beta, calls_b2, width = 4)

# Step 5: Resolve alpha and beta overlaps
avg_a = calcRegionAverage(alpha, calls_a3) 
avg_b = calcRegionAverage(beta, calls_b3)
diff_a = [avg_a[i] - avg_b[i] for i in range(len(avg_a))] 
diff_b = [avg_b[i] - avg_a[i] for i in range(len(avg_a))] 
calls_a4 = checkSupport(calls_a3, diff_a)
calls_b4 = checkSupport(calls_b3, diff_b)

print("Step 5")
print(makesstr(calls_a4, 'H'))
print(makesstr(calls_b4, 'E'))

print("Final prediction")
# Combine calls and replace remaining '-' symbols to C (coil)
prediction = predicted_sequence = ''.join(['H' if alpha else ('E' if beta else 'C') for (alpha, beta) in zip(calls_a4, calls_b4)])
print(prediction)

Steps 1 to 4
-HHHHHHHHHHHHHHHHHHHHH---
---------EEEEEE----------
Step 5
-HHHHHHHHHHHHHHHHHHHHHH--
-------------------------
Final prediction
CHHHHHHHHHHHHHHHHHHHHHHCC


#### Exercise 5 : Calculating baseline accuracy Outline
What percent accuracy would you expect if you were guessing the assignment of 3-class (H, E or C) secondary structure at random, assuming that each class is equally likely?



#### Exercise 6 : Assessing accuracy for a specific protein

In [6]:
# store protein and structure sequences in dictionary for easy retrieval
protein_map = {}
structure_map = {}

for protein in prot2:
    protein_map.update({protein.name:protein})
    
for structure in sstr3:
    structure_map.update({structure.name:structure})
    
protein = protein_map.get("1EVH")
structure = structure_map.get("1EVH")
print("protein: ", protein)
print("structure: ", structure)

protein:  1EVH: SEQSICQARAAVMVYDDANKKWVPAGGSTGFSRVHIYHHTGNNTFRVVGRKIQDHQVVINCAIPKGLKYNQATQTFHQWRDARQVYGLNFGSKEDANVFASAMMHALEVLN
structure:  1EVH: CEEEEEEEEEEEEEEECCCCEEEEHHHCCCCEEEEEEEECCCCEEEEEEEECCCCCEEEEEEECCCCCCECCCCCEEEEECCCCEEEEEECCHHHHHHHHHHHHHHHHHHC


In [7]:
print("Steps 1 to 4")
# Step 2: Alpha-helix 
alpha = getScores(protein, 0) # values from column 0
beta = getScores(protein, 1) # values from column 1
calls_a1 = markCountAbove(alpha, width=6, call_cnt=4)
print(makesstr(calls_a1, 'H'))

calls_a2 = extendDownstream(alpha, calls_a1, width=4)
calls_a3 = extendUpstream(alpha, calls_a2, width=4)

calls_b1 = markCountAbove(beta, width=5, call_cnt=3)
print(makesstr(calls_b1, 'E'))
calls_b2 = extendDownstream(beta, calls_b1, width=4)
calls_b3 = extendUpstream(beta, calls_b2, width=4)

avg_a = calcRegionAverage(alpha, calls_a3)
avg_b = calcRegionAverage(beta, calls_b3)

Steps 1 to 4
-HHHHHHHHHHHHHHHHHHHHHHHHH---------------HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH----HHHHHHHHHHHHHHHHHHHHHHHHHHHH
-EEEEEEEEEEEEEEEE---------------EEEEEEEE--EEEEEEE--EEEEEEEEEEEE---EEEEEEEEEEEEEE-EEEEEEEEE------------EEEEEEEEE


In [8]:
diff_a = [avg_a[i] - avg_b[i] for i in range(len(avg_a))]
diff_b = [avg_b[i] - avg_a[i] for i in range(len(avg_a))]

print("Step 5")
calls_a4 = checkSupport(calls_a3, diff_a)
calls_b4 = checkSupport(calls_b3, diff_b)
print(makesstr(calls_a4, 'H'))
print(makesstr(calls_b4, 'E'))

print("Final prediction")
# Combine calls and replace remaining '-' symbols to C (coil)
prediction = predicted_sequence = ''.join(['H' if alpha else ('E' if beta else 'C') for (alpha, beta) in zip(calls_a4, calls_b4)])
print(prediction)

Step 5
H-----------------HHHHHHHH---------------H----------------------H------------------HHHHHHHHHHHHHHHHHHHHHHHHHHHH
-EEEEEEEEEEEEEEEEE------------EEEEEEEEEE--EEEEEEEEEEEEEEEEEEEEEE-EEEEEEEEEEEEEEEEEE----------------------------
Final prediction
HEEEEEEEEEEEEEEEEEHHHHHHHHCCCCEEEEEEEEEECHEEEEEEEEEEEEEEEEEEEEEEHEEEEEEEEEEEEEEEEEEHHHHHHHHHHHHHHHHHHHHHHHHHHHH


In [9]:
myPrediction = prediction
mySS = [sequence for sequence in sstr3 if sequence.name == "1EVH"][0]

index = 15 #position to look at
if myPrediction[index] == mySS[index]:
    print(True)

True


overall accuracy of Chou-Fasman for predicting secondary structure

In [10]:
# to check the accuracy we simply compare our prediction to the one obtained from sstr3.fa
position = 0
match_count = 0
for element in structure.sequence:
    if element == "H" and calls_a4[position]:
        match_count += 1 # matched alpha helix
    if element == "E" and calls_b4[position]:
        match_count += 1 # matched beta sheet
    if element != "H" and element !="E" and not(calls_a4[position]) and not(calls_b4[position]):
        match_count += 1 # matched other
    position += 1
        
print(position, " structures with ", match_count, " correctly matched.")
print("Accuracy %.2f%%" % ((float(match_count) /  position) * 100))

111  structures with  68  correctly matched.
Accuracy 61.26%


percent accuracy for alpha-helix predictions

In [11]:
position = 0
tp = 0  # number of true positives (correctly identified calls)
tn = 0  # number of true negatives (correctly missed no-calls)
fp = 0  # number of false positives (incorrectly identified no-calls)
fn = 0  # number of false negatives (incorrectly missed calls)
for element in structure.sequence:
    if element == "H" and calls_a4[position]:
        tp += 1 # matched alpha helix
    if element != "H" and not(calls_a4[position]):
        tn += 1 # matched beta sheet
    if element != "H" and calls_a4[position]:
        fp += 1 # matched beta sheet
    if element == "H" and not(calls_a4[position]):
        fn += 1 # matched beta sheet
    position += 1

print("alpha-helix Accuracy %.2f%%" % ((float((tp+tn) /  (tp+tn+fp+fn)) * 100)))

alpha-helix Accuracy 81.98%


percent accuracy for beta-strands predictions

In [12]:
position = 0
tp = 0  # number of true positives (correctly identified calls)
tn = 0  # number of true negatives (correctly missed no-calls)
fp = 0  # number of false positives (incorrectly identified no-calls)
fn = 0  # number of false negatives (incorrectly missed calls)
for element in structure.sequence:
    if element == "E" and calls_b4[position]:
        tp += 1 # matched alpha helix
    if element != "E" and not(calls_b4[position]):
        tn += 1 # matched beta sheet
    if element != "E" and calls_b4[position]:
        fp += 1 # matched beta sheet
    if element == "E" and not(calls_b4[position]):
        fn += 1 # matched beta sheet
    position += 1

print("beta-strand Accuracy %.2f%%" % ((float((tp+tn) /  (tp+tn+fp+fn)) * 100)))

beta-strand Accuracy 70.27%


In [28]:
atp = 0  # number of true positives (correctly identified calls)
atn = 0  # number of true negatives (correctly missed no-calls)
afp = 0  # number of false positives (incorrectly identified no-calls)
afn = 0  # number of false negatives (incorrectly missed calls)

btp = 0
btn = 0
bfp = 0
bfn = 0

ctp = 0
ctn = 0
cfp = 0
cfn = 0
tp = 0
tn = 0
fp = 0
fn = 0

for index in range(5):
    protein = prot2[index]
    structure = sstr3[index]
#     print("protein: ", protein)
#     print("structure: ", structure)
    
#     print("Steps 1 to 4")
    # Step 2: Alpha-helix
    alpha = getScores(protein, 0) # values from column 0
    beta = getScores(protein, 1) # values from column 1
    calls_a1 = markCountAbove(alpha, width=6, call_cnt=4)
#     print(makesstr(calls_a1, 'H'))
    
    calls_a2 = extendDownstream(alpha, calls_a1, width=4)
    calls_a3 = extendUpstream(alpha, calls_a2, width=4)

    calls_b1 = markCountAbove(beta, width=5, call_cnt=3)
#     print(makesstr(calls_b1, 'E'))
    calls_b2 = extendDownstream(beta, calls_b1, width=4)
    calls_b3 = extendUpstream(beta, calls_b2, width=4)

    avg_a = calcRegionAverage(alpha, calls_a3)
    avg_b = calcRegionAverage(beta, calls_b3)

    diff_a = [avg_a[i] - avg_b[i] for i in range(len(avg_a))]
    diff_b = [avg_b[i] - avg_a[i] for i in range(len(avg_a))]
    
#     print("Step 5")
    calls_a4 = checkSupport(calls_a3, diff_a)
    calls_b4 = checkSupport(calls_b3, diff_b)
#     print(makesstr(calls_a4, 'H'))
#     print(makesstr(calls_b4, 'E'))
    
#     print("Final prediction")
    # Combine calls and replace remaining '-' symbols to C (coil)
    prediction = predicted_sequence = ''.join(['H' if alpha else ('E' if beta else 'C') for (alpha, beta) in zip(calls_a4, calls_b4)])
#     print(prediction)
    
    position = 0
    for element in structure.sequence:
        if element == "H" and calls_a4[position]:
            atp += 1 # matched alpha helix
        if element != "H" and not(calls_a4[position]):
            atn += 1 # matched beta sheet
        if element != "H" and calls_a4[position]:
            afp += 1 # matched beta sheet
        if element == "H" and not(calls_a4[position]):
            afn += 1 # matched beta sheet
        position += 1
        
    position = 0
    for element in structure.sequence:
        if element == "E" and calls_b4[position]:
            btp += 1 # matched alpha helix
        if element != "E" and not(calls_b4[position]):
            btn += 1 # matched beta sheet
        if element != "E" and calls_b4[position]:
            bfp += 1 # matched beta sheet
        if element == "E" and not(calls_b4[position]):
            bfn += 1 # matched beta sheet
        position += 1
        
    position = 0
    for element in structure.sequence:
        if element == "C" and not(calls_a4[position]) and not(calls_b4[position]):
            ctp += 1 # matched alpha helix
        if element != "C" and ((calls_a4[position]) or (calls_b4[position])):
            ctn += 1 # matched beta sheet
        if element != "C" and not(calls_a4[position]) and not(calls_b4[position]):
            cfp += 1 # matched beta sheet
        if element == "C" and ((calls_a4[position]) or (calls_b4[position])):
            cfn += 1 # matched beta sheet
        position += 1
        
    tp = atp+btp+ctp
    tn = atn+btn+ctn
    fp = afp+bfp+cfp
    fn = afn+bfn+cfn

####### Accuracy calculations #######
print("alpha-helices True  Positive = %d" % atp)
print("alpha-helices True  Negative = %d" % atn)
print("alpha-helices False Positive = %d" % afp)
print("alpha-helices False Negative = %d" % afn)
print("alpha-helix Accuracy %.2f%%" % ((float((atp+atn) /  (atp+atn+afp+afn)) * 100)))

print("beta-strands True  Positive = %d" % btp)
print("beta-strands True  Negative = %d" % btn)
print("beta-strands False Positive = %d" % bfp)
print("beta-strands False Negative = %d" % bfn)
print("beta-strands Accuracy %.2f%%" % ((float((btp+btn) /  (btp+btn+bfp+bfn)) * 100)))

print("coil True  Positive = %d" % ctp)
print("coil True  Negative = %d" % ctn)
print("coil False Positive = %d" % cfp)
print("coil False Negative = %d" % cfn)
print("coil Accuracy %.2f%%" % ((float((ctp+ctn) /  (ctp+ctn+cfp+cfn)) * 100)))

print("True  Positive = %d" % tp)
print("True  Negative = %d" % tn)
print("False Positive = %d" % fp)
print("False Negative = %d" % fn)
print("Accuracy %.2f%%" % ((float((tp+tn) /  (tp+tn+fp+fn)) * 100)))

alpha-helices True  Positive = 213
alpha-helices True  Negative = 353
alpha-helices False Positive = 196
alpha-helices False Negative = 150
alpha-helix Accuracy 62.06%
beta-strands True  Positive = 107
beta-strands True  Negative = 418
beta-strands False Positive = 281
beta-strands False Negative = 106
beta-strands Accuracy 57.57%
coil True  Positive = 81
coil True  Negative = 542
coil False Positive = 34
coil False Negative = 255
coil Accuracy 68.31%
True  Positive = 401
True  Negative = 1313
False Positive = 511
False Negative = 511
Accuracy 62.65%


In [30]:
(62.06 + 57.57 + 68.31)/3

62.64666666666667